# 🤖 Nano.Ai APK Builder

Build the Nano.Ai Android APK directly in Google Colab!

This notebook will:
1. Install Android SDK & NDK
2. Clone the repository with llama.cpp
3. Build the APK
4. Download the APK

**Runtime:** ~15-20 minutes on first run

## Step 1: Install Java & Android SDK

In [ ]:
# Install Java 17 (required for Android Gradle)
!apt-get update -qq
!apt-get install -qq openjdk-17-jdk-headless > /dev/null 2>&1

# Set JAVA_HOME
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'
os.environ['PATH'] = f"{os.environ['JAVA_HOME']}/bin:{os.environ['PATH']}"

# Verify Java
!java -version

In [ ]:
# Install Android SDK command-line tools
!mkdir -p /root/android-sdk/cmdline-tools
%cd /root/android-sdk/cmdline-tools

# Download command-line tools
!wget -q https://dl.google.com/android/repository/commandlinetools-linux-11076708_latest.zip -O cmdtools.zip
!unzip -q -o cmdtools.zip
!mv cmdline-tools latest 2>/dev/null || true
!rm cmdtools.zip

# Set environment variables
os.environ['ANDROID_HOME'] = '/root/android-sdk'
os.environ['ANDROID_SDK_ROOT'] = '/root/android-sdk'
os.environ['PATH'] = f"/root/android-sdk/cmdline-tools/latest/bin:/root/android-sdk/platform-tools:{os.environ['PATH']}"

print("✅ Android SDK tools installed")

In [ ]:
# Accept licenses and install required SDK components
!yes | sdkmanager --licenses > /dev/null 2>&1

# Install required components
!sdkmanager "platforms;android-34" "build-tools;34.0.0" "platform-tools" --quiet
!sdkmanager "ndk;25.2.9519653" "cmake;3.22.1" --quiet

# Set NDK path
os.environ['ANDROID_NDK_HOME'] = '/root/android-sdk/ndk/25.2.9519653'

print("✅ Android SDK components installed")
print(f"   - Platform: android-34")
print(f"   - Build Tools: 34.0.0")
print(f"   - NDK: 25.2.9519653")
print(f"   - CMake: 3.22.1")

## Step 2: Clone Repository

In [ ]:
# Clone Nano.Ai repository
%cd /content
!rm -rf NanoAi 2>/dev/null || true
!git clone https://github.com/mithun50/NanoAi.git
%cd NanoAi

print("\n✅ Repository cloned")

In [ ]:
# Clone llama.cpp submodule
%cd /content/NanoAi

# Add llama.cpp as submodule if not exists
!if [ ! -d "app/src/main/cpp/llama.cpp" ]; then \
    git clone --depth 1 https://github.com/ggerganov/llama.cpp.git app/src/main/cpp/llama.cpp; \
fi

print("✅ llama.cpp cloned")

In [ ]:
# Create local.properties with SDK path
with open('/content/NanoAi/local.properties', 'w') as f:
    f.write(f"sdk.dir={os.environ['ANDROID_HOME']}\n")
    f.write(f"ndk.dir={os.environ['ANDROID_NDK_HOME']}\n")

print("✅ local.properties created")

## Step 3: Build APK

In [ ]:
# Make gradlew executable and build
%cd /content/NanoAi

# Download gradle wrapper if not exists
!if [ ! -f "gradlew" ]; then \
    gradle wrapper --gradle-version 8.4; \
fi

!chmod +x gradlew

print("\n🔨 Starting build... (this may take 10-15 minutes)\n")

In [ ]:
# Build debug APK
%cd /content/NanoAi
!./gradlew assembleDebug --no-daemon --stacktrace

print("\n✅ Build complete!")

## Step 4: Download APK

In [ ]:
# List generated APKs
import glob

apks = glob.glob('/content/NanoAi/app/build/outputs/apk/**/*.apk', recursive=True)

print("📦 Generated APKs:")
for apk in apks:
    size_mb = os.path.getsize(apk) / (1024 * 1024)
    print(f"   - {os.path.basename(apk)} ({size_mb:.1f} MB)")

In [ ]:
# Download APK
from google.colab import files

# Find the main debug APK
apk_path = '/content/NanoAi/app/build/outputs/apk/debug/app-debug.apk'

if os.path.exists(apk_path):
    # Rename for clarity
    final_name = 'NanoAi-debug.apk'
    !cp {apk_path} /content/{final_name}
    
    print(f"\n📥 Downloading {final_name}...\n")
    files.download(f'/content/{final_name}')
else:
    # Try to find any APK
    if apks:
        print(f"\n📥 Downloading {os.path.basename(apks[0])}...\n")
        files.download(apks[0])
    else:
        print("❌ No APK found. Check build logs above for errors.")

## Optional: Build Release APK

To build a signed release APK, you need to provide a keystore.

In [ ]:
# Generate a debug keystore for release build (optional)
# Uncomment to use:

# !keytool -genkey -v -keystore /content/debug.keystore \
#     -storepass android -alias androiddebugkey \
#     -keypass android -keyalg RSA -keysize 2048 -validity 10000 \
#     -dname "CN=Debug, OU=Debug, O=Debug, L=Debug, ST=Debug, C=US"

# !./gradlew assembleRelease \
#     -Pandroid.injected.signing.store.file=/content/debug.keystore \
#     -Pandroid.injected.signing.store.password=android \
#     -Pandroid.injected.signing.key.alias=androiddebugkey \
#     -Pandroid.injected.signing.key.password=android

---

## 📱 Installation

1. Transfer the APK to your Android device
2. Enable "Install from unknown sources" in settings
3. Open the APK to install
4. Download a GGUF model and import it into the app

**Recommended Models:**
- TinyLlama 1.1B Q4_K_M (~700MB) - Fast, low memory
- Qwen 2.5 3B Q4_K_M (~2GB) - Good balance
- Phi-2 Q4_K_M (~1.6GB) - Good quality

---

Made with ❤️ using llama.cpp